In [1]:
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import bessel_approx_np, bessel_approx_tf
from utils.losses import von_mises_log_likelihood_np, von_mises_log_likelihood_tf
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [3]:
#x_tf = tf.placeholder(tf.float32, shape=[None, 3, image_width, image_height])
mu_pred_tf = tf.placeholder(tf.float32, shape=[None, 2])
log_kappa_tf = tf.placeholder(tf.float32, shape=[None, 1])
y_true_tf = tf.placeholder(tf.float32, shape=[None, 2])
y_pred_tf = tf.concat([mu_pred_tf, log_kappa_tf], axis=1)

In [4]:
def von_mises_neg_log_likelihood_keras(y_true, y_pred):
    mu_pred = y_pred[:,0:2]
    log_kappa_pred = y_pred[:, 2:]
    return -von_mises_log_likelihood_tf(y_true, mu_pred, log_kappa_pred, input_type='biternion')

In [5]:
loss1= -von_mises_log_likelihood_tf(y_true_tf, mu_pred_tf, log_kappa_tf, input_type='biternion')
loss2 = von_mises_neg_log_likelihood_keras(y_true_tf, y_pred_tf)

In [6]:
log_kappa = np.ones([ytr_bit.shape[0], 1])*0

In [7]:
with tf.Session() as sess:
    log_likelighood1 = sess.run(loss1, feed_dict={y_true_tf:ytr_bit[0:100], mu_pred_tf:ytr_bit[0:100], log_kappa_tf:log_kappa[0:100]})
    log_likelighood2 = sess.run(loss2, feed_dict={y_true_tf:ytr_bit[0:100], mu_pred_tf:ytr_bit[0:100], log_kappa_tf:log_kappa[0:100]})

In [8]:
log_likelighood1

1.0737537

In [9]:
log_likelighood2

1.0737537

In [10]:
von_mises_log_likelihood_np(ytr_bit[0:100], ytr_bit[0:100], log_kappa[0:100], input_type='biternion')

-1.0737913709802149

In [14]:
X = Input(shape=[50,50,3])
vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

In [15]:
mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))
log_sigma = Input(shape=[1]) #Dense(1)(vgg_x)
mu_log_sigma = concatenate([mu, log_sigma])

In [16]:
biternion_vgg_prob = Model([X, log_sigma], mu_log_sigma)

In [22]:
biternion_vgg_prob.input

[<tf.Tensor 'input_2:0' shape=(?, 50, 50, 3) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(?, 1) dtype=float32>]

In [23]:
biternion_vgg_prob.compile(loss=von_mises_neg_log_likelihood_keras,
                           optimizer='adam')

In [38]:
biternion_vgg_prob.fit(x=[xtr, log_kappa] , y=ytr_bit, batch_size=100, validation_split=0.1, epochs=50)

Train on 7002 samples, validate on 778 samples
Epoch 1/50
7002/7002 [==============================] - 48s - loss: 1.3098 - val_loss: 1.6380
Epoch 2/50
7002/7002 [==============================] - 48s - loss: 1.2888 - val_loss: 1.5227
Epoch 3/50
7002/7002 [==============================] - 48s - loss: 1.2764 - val_loss: 1.5227
Epoch 4/50
7002/7002 [==============================] - 48s - loss: 1.3225 - val_loss: 1.7930
Epoch 5/50
7002/7002 [==============================] - 47s - loss: 1.3089 - val_loss: 1.5553
Epoch 6/50
7002/7002 [==============================] - 48s - loss: 1.2985 - val_loss: 1.4316
Epoch 7/50
7002/7002 [==============================] - 48s - loss: 1.2698 - val_loss: 1.4393
Epoch 8/50
7002/7002 [==============================] - 48s - loss: 1.2713 - val_loss: 1.4670
Epoch 9/50
7002/7002 [==============================] - 47s - loss: 1.2618 - val_loss: 1.4612
Epoch 10/50
7002/7002 [==============================] - 47s - loss: 1.2632 - val_loss: 1.5616
Epoch 11/50


In [29]:
log_kappa_te = np.ones([ytr_bit.shape[0], 1])*0

In [39]:
yte_preds = bit2deg(biternion_vgg_prob.predict([xte, log_kappa_te])[:,0:2])

In [40]:
loss = maad_from_deg(yte_preds, yte_deg)

In [41]:
np.mean(loss)

30.179481105679233